In [1]:
import pandas as pd

import requests  # 크롤링을 위한 라이브러리
from bs4 import BeautifulSoup  # 파싱을 위한 라이브러리

 http://02.the114.net/?PhoneNumber=023376969

In [2]:
pharmacy = pd.read_csv('seoul(english).csv')

In [3]:
pharmacy.head()

,"﻿""키값""",약국이름,행정시,행정구,행정동,전화번호
0,BE_LiST22-0102,Saegwang Pharmacy,Seoul,Dongdaemun-gu,Jegi-dong,962-8989
1,BE_LiST22-0103,Bokji Pharmacy,Seoul,Dongdaemun-gu,Dapsimni1-dong,2217-3606
2,BE_LiST22-0104,Chamewha Pharmacy,Seoul,Dongdaemun-gu,Dapsimni1-dong,2247-5029
3,BE_LiST22-0105,Janganpyeong Pharmacy,Seoul,Dongdaemun-gu,Jangan2-dong,2212-9430
4,BE_LiST22-0106,Saeewha Pharmacy,Seoul,Dongdaemun-gu,Jangan2-dong,2214-2488


In [128]:
pharmacy2 = pd.read_csv("seoul(korean).csv")

In [129]:
pharmacy2['전화번호'] = "02"+pharmacy2['전화번호']
pharmacy2['전화번호'] = pharmacy2['전화번호'].str.replace("-","")

In [4]:
tel = pharmacy['전화번호']

In [7]:
tel[:5]

0     962-8989
1    2217-3606
2    2247-5029
3    2212-9430
4    2214-2488
Name: 전화번호, dtype: object

In [10]:
tel = "02" + tel

In [11]:
tel[:5]

0     02962-8989
1    022217-3606
2    022247-5029
3    022212-9430
4    022214-2488
Name: 전화번호, dtype: object

In [25]:
tel = tel.str.replace("-","")

In [26]:
tel[:5]

0     029628989
1    0222173606
2    0222475029
3    0222129430
4    0222142488
Name: 전화번호, dtype: object

In [ ]:
pharmacy['전화번호'] = "02"+pharmacy['전화번호']
pharmacy['전화번호'] = pharmacy['전화번호'].str.replace("-","")

In [29]:
response = requests.get("http://02.the114.net/?PhoneNumber=023376969")
dom = BeautifulSoup(response.content, "html.parser")

In [31]:
tel_elements = dom.select('body')

In [37]:
contents = tel_elements[0].attrs.get('onload')

In [42]:
contents = contents.replace("initializeMap(","")

In [124]:
list = []

In [125]:
for x in tel:
    url = "http://02.the114.net/?PhoneNumber=" + x
    response = requests.get(url)
    dom = BeautifulSoup(response.content, "html.parser")
    tel_elements = dom.select('body')
    contents = tel_elements[0].attrs.get('onload')
    if contents:
        contents = contents.replace("initializeMap(","")
        tmp = contents.split(",")[0:2]
        tmp.append(x)
        list.append(tmp)

In [126]:
list[:10]

[['37.5794511', '127.0421273', '029628989'],
 ['37.5705108', '127.0481395', '0222173606'],
 ['37.565682', '127.054819', '0222475029'],
 ['37.571577', '127.074285', '0222129430'],
 ['37.5729167', '127.072075', '0222142488'],
 ['37.5676501', '127.0722409', '0222452594'],
 ['37.57449', '127.0387234', '029641717'],
 ['37.5740822', '127.058008', '0222159454'],
 ['37.578063', '127.057294', '0222156291'],
 ['37.5799521', '127.0662796', '0222121931']]

In [130]:
pharmacy2[pharmacy2['전화번호']=='029628989']

,"﻿""키값""",약국이름,주소,도로명주소,행정시,행정구,행정동,전화번호,가능외국어
349,BE_LiST22-0102,세광약국,NaN,동대문구 왕산로 171(제기동),서울특별시,동대문구,제기동,029628989,"영어,일어"


In [133]:
tel_df = pd.DataFrame(list,columns=['latitude','longtitude','전화번호'])

In [134]:
tel_df.head()

,latitude,longtitude,전화번호
0,37.5794511,127.0421273,029628989
1,37.5705108,127.0481395,0222173606
2,37.565682,127.054819,0222475029
3,37.571577,127.074285,0222129430
4,37.5729167,127.072075,0222142488


In [135]:
df = pd.merge(pharmacy, tel_df, on='전화번호')

In [142]:
df.head()

,"﻿""키값""",약국이름,행정시,행정구,행정동,전화번호,latitude,longtitude
0,BE_LiST22-0102,Saegwang Pharmacy,Seoul,Dongdaemun-gu,Jegi-dong,029628989,37.5794511,127.0421273
1,BE_LiST22-0103,Bokji Pharmacy,Seoul,Dongdaemun-gu,Dapsimni1-dong,0222173606,37.5705108,127.0481395
2,BE_LiST22-0104,Chamewha Pharmacy,Seoul,Dongdaemun-gu,Dapsimni1-dong,0222475029,37.565682,127.054819
3,BE_LiST22-0105,Janganpyeong Pharmacy,Seoul,Dongdaemun-gu,Jangan2-dong,0222129430,37.571577,127.074285
4,BE_LiST22-0106,Saeewha Pharmacy,Seoul,Dongdaemun-gu,Jangan2-dong,0222142488,37.5729167,127.072075


In [141]:
df.to_csv("seoul(english) with map.csv")

## 지도 그리기 
 - https://www.lucypark.kr/blog/2015/06/24/seoul-matzip-mapping
 - 참조 
 